In [ ]:
mport pandas as pd
import numpy as np
import math
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Bidirectional, LSTM, Flatten, Dense,LayerNormalization
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras import backend as K
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from numpy import argmax
from tensorflow.keras.layers import MultiHeadAttention

def preprocess_data(df):
    # Remove columns where their last row is null
    df = df.drop(columns=df.columns[df.iloc[-1].isnull()])

    # Remove columns with more than 80% NaN values and fill others with mean
    threshold = 0.8 * len(df)
    df = df.dropna(thresh=threshold, axis=1)
    df = df.fillna(df.mean())
    
    # Pad columns to have 496 rows, with last row unchanged
    padding_len = 497 - len(df)
    padding = pd.DataFrame(0, index=np.arange(padding_len), columns=df.columns)
    df = pd.concat([padding, df], axis=0)
    
    return df

def load_data(*files):
    dataframes = []

    for file in files:
        # Load dataframe
        df = pd.read_csv(file, header=0)
        
        # Preprocess the data
        df = preprocess_data(df)
        # Add a prefix to each column name based on the file name
        prefix = file.split('.')[0]  # Assuming the file name is '11_2016.csv', this gets '11_2016'
        df.columns = [f"{prefix}_{col}" for col in df.columns]
        
        # Reset index after preprocessing to ensure unique indices
        df.reset_index(drop=True, inplace=True)

        dataframes.append(df)

    # Concatenate all preprocessed dataframes
    result = pd.concat(dataframes, axis=1)

    return result



# Build the LSTM model for multi-class classification
def build_classifier_model(input_shape, num_classes):
    model = Sequential()
    model.add(Bidirectional(LSTM(units=290, input_shape=(1, 496), return_sequences=True)))     
    #model.add(MultiHeadAttention(num_heads=2, key_dim=290))
    #model.add(LayerNormalization(epsilon=1e-6))  # Layer normalization can help stabilize the outputs
    # Add another LSTM layer with 120 units
    model.add(LSTM(120, return_sequences=True))
    
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))  # softmax for multi-class
    model.compile(optimizer='rmsprop', loss="categorical_crossentropy", metrics=['accuracy', f1_score])
    return model

# F1 Score Custom Metric
def f1_score(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    precision_val = precision(y_true, y_pred)
    recall_val = recall(y_true, y_pred)
    return 2 * ((precision_val * recall_val) / (precision_val + recall_val + K.epsilon()))




# Main script execution
if __name__ == '__main__':
    # Load data
    # Load data
    files = ['11_2016.csv', '12_2016.csv', '01_2017.csv', '02_2017.csv', '03_2018.csv',
         '12_2017.csv', '01_2018.csv', '02_2018.csv', '03_2018.csv']
    df = load_data(*files)
    
    # Handle missing values, for example, by replacing them
    #df.fillna(method='ffill', inplace=True)  # Forward fill
    # Assuming single_file_result is your dataframe

# Get unique values from row 496
    unique_values_row_496 = df.iloc[496].unique()

# Filter out the expected values
    unexpected_values = [value for value in unique_values_row_496 if value not in [-3, -2, -1, 0, 1, 2, 3]]

    print(df.tail())
    df = df.T
    caler2 = StandardScaler()
    # Use the initial 200 rows for training
    training_set = df.iloc[:220, :]
    X_train = training_set.iloc[:, :-1].values
    X_train = scaler2.fit_transform(X_train)
    y_train = training_set.iloc[:, -1].values
    # Reshape the X_train
    num_samples_train, num_features_train = X_train.shape
    X_train = np.reshape(X_train, (num_samples_train, 1, num_features_train))
    # Prepare the testing set, using the remaining rows (from 200 to 268)
    testing_set = df.iloc[220:268, :-1]
    X_test = scaler2.fit_transform(testing_set)

    
    y_test = df.iloc[220:268, -1].values
    #   Reshape the X_test
    num_samples_test = X_test.shape[0]
    X_test = np.reshape(X_test, (num_samples_test, 1, 496))
    # Convert y_train and y_test to categorical
    num_classes = len(np.unique(y_train))
    y_train = y_train.astype(int)
    num_classes2 = len(np.unique(y_test))
    y_test = y_test.astype(int)
    # Convert labels to categorical
    y_train = to_categorical(y_train, num_classes=7)  # Assuming 5 classes
    y_test = to_categorical(y_test, num_classes=7)
    # Train the model
    model = build_classifier_model(496,7) # Assuming you have a function called build_classifier_model
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=40, batch_size=64)

    # You can collect metrics or save models, weights, etc., during/after each iteration if required.

    loss, accuracy, f1score = model.evaluate(X_test, y_test, verbose=0)
    print('Test Accuracy: %.2f%%' % (accuracy * 100))
    print('Test F1 Score: %.2f' % f1score)
   
